In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

In [97]:
path = r'D:\NUS\project\LTA_data\od_comuting_flow\train\2022'

In [98]:
szone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
lis_szone = list(set(szone['SUBZONE_N']))
print('the number of subzone is: {}'.format(len(lis_szone)))
lis_file = os.listdir(path=path)
lis_file

the number of subzone is: 332


['train_commuting_flow_202201.csv',
 'train_commuting_flow_202202.csv',
 'train_commuting_flow_202203.csv',
 'train_commuting_flow_202204.csv']

In [7]:
def get_raw_data(path,i):
    lis_file = os.listdir(path=path)
    
    df = pd.read_csv(path+'\\'+lis_file[i])
    
    return df

In [8]:
def aggregate_od(od):
    origin,destination,total_trip = 'Origin_Subzone','Destination_Subzone','TOTAL_TRIPS'
    df = od.groupby([origin,destination]).sum(total_trip).reset_index()
    return df

In [9]:
def creat_szone_matrix(lis):
    cols = ['SUBZONE_N']
    for i in lis:
        cols.append(i)
    szone_matrix = pd.DataFrame(columns = cols)
    szone_matrix['SUBZONE_N'] = lis
    
    return szone_matrix

In [10]:
def output_destination(subzone,df):
    dataframe = df.loc[df['Origin_Subzone'] == subzone].copy()
    dataframe.index = range(len(dataframe))
    lis_destination = dataframe.loc[:,'Destination_Subzone']
    lis_total_trip = dataframe.loc[:,'TOTAL_TRIPS']
    
    return list(lis_destination),list(lis_total_trip)

In [11]:
def commuting_flow_matching(szone_m,subzone,lis_sz,lis_trip):
    index = szone_m.loc[szone_m['SUBZONE_N'] == subzone].index
    for i in range(len(lis_szone)):
        for x in range(len(lis_sz)):
            if lis_sz[x] == lis_szone[i]:
                szone_m.loc[index,lis_szone[i]] = lis_trip[x]               
    return szone_m

In [104]:
def main():
    path = r'D:\NUS\project\LTA_data\od_comuting_flow\train\2022'
    szone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
    lis_szone = list(set(szone['SUBZONE_N']))
    lis_szone.sort()
    
    lis_file = os.listdir(path=path)
    print(lis_file[2])
    
    commuting_flow = get_raw_data(path,2)
    aggregate_commuting_flow = aggregate_od(commuting_flow)
    szone_matrix = creat_szone_matrix(lis_szone)
    
    for i in lis_szone:
        lis_d,lis_trip = output_destination(i,aggregate_commuting_flow)
        szone_matrix = commuting_flow_matching(szone_matrix,i,lis_d,lis_trip)
        
    szone_matrix = szone_matrix.fillna(0)
    return szone_matrix

In [105]:
if __name__ == '__main__':
    train_szone_commuting_matrix = main()

train_commuting_flow_202203.csv


In [94]:
result_path = r'D:\NUS\project\LTA_data\commuting_flow_matrix\train'

In [106]:
train_szone_commuting_matrix.to_csv(result_path+'\\'+'train_commuting_flow_matrix_202203_by_subzone.csv',index = False)

In [107]:
train_szone_commuting_matrix

,SUBZONE_N,ADMIRALTY,AIRPORT ROAD,ALEXANDRA HILL,ALEXANDRA NORTH,ALJUNIED,ANAK BUKIT,ANCHORVALE,ANG MO KIO TOWN CENTRE,ANSON,...,YIO CHU KANG EAST,YIO CHU KANG NORTH,YIO CHU KANG WEST,YISHUN CENTRAL,YISHUN EAST,YISHUN SOUTH,YISHUN WEST,YUHUA EAST,YUHUA WEST,YUNNAN
0,ADMIRALTY,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIRPORT ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALEXANDRA HILL,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALEXANDRA NORTH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALJUNIED,0,0,0,0,9511,3304,876,0,0,...,0,0,0,0,0,0,9822,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,YISHUN SOUTH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
328,YISHUN WEST,0,0,0,0,9475,635,42,0,0,...,0,0,0,0,0,0,0,0,0,0
329,YUHUA EAST,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,YUHUA WEST,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
